In [32]:
import pandas as pd
import pickle
import re

In [ ]:
### Carga df desde excel
path_xlsx = "/Users/mac2/Library/CloudStorage/GoogleDrive-edwar.perez@vlex.com/Mi unidad/EP_Coding_Vlex/Martindale/datos_abogados.xlsx"
df_0 = pd.read_excel(path_xlsx)
df_0 = df_0.dropna(axis=1, how='all')
df_0.iloc[:,9:].head(2)
#df_0.head(2)


In [35]:
### Carga del df desde un archivo .pkl
with open("data_attornies.pkl", "rb") as f:
    data_for_PA = pickle.load(f) # Carga de lista de diccionarios de abogados de primera iteración, ya la lista no es 0.
    # Esta variable va a ser mas general por lo que se va a emplear mas adelante. o asignar a la primera corrida. 
data_for_PA
df = pd.DataFrame(data_for_PA)
df = df.where(pd.notnull(df), None)
df = df.dropna(axis=1, how='all')
df = df.drop_duplicates()
df.drop(['Education & Credentials'], axis=1, inplace=True)
df.tail(5)


,url_martindale,name,firm,url_firm_martindale,website_firm,Biography,Areas of Practice,Contact Information,University Attended,Law School Attended,...,"Other Practice Areas in Guayama, Puerto Rico","Other Practice Areas in Utuado, Puerto Rico","Other Practice Areas in Fajardo, Puerto Rico",Special Agencies,"Other Practice Areas in San German, Puerto Rico","Other Practice Areas in Cayey, Puerto Rico","Other Practice Areas in Luquillo, Puerto Rico",Responsibilities,"Other Practice Areas in Toa Baja, Puerto Rico","Other Practice Areas in Miami, FL"
2672,https://www.martindale.com/attorney/soledad-re...,Soledad Reichard-Megwinoff,None,None,None,None,Employee Benefits,None,"Boston College, B.S., 1985","University of Puerto Rico, J.D., 1991",...,None,None,None,None,None,None,None,None,None,None
2673,https://www.martindale.com/attorney/tatiana-le...,Tatiana Leal-Gonzalez,Jackson Lewis P.C.,None,None,None,"Disability, Leave and Health Management, Emplo...",None,"University of Puerto Rico, B.A., 2010","University of Puerto Rico Law School, J.D., 2013",...,None,None,None,None,None,None,None,None,None,None
2674,https://www.martindale.com/attorney/miguel-a-c...,Miguel A. Céceres-Colón,None,None,None,None,"Estate Law, Corporate Law",None,"University of Puerto Rico, B.B.A., 1997","University of Puerto Rico, J.D., 2000",...,None,None,None,None,None,None,None,None,None,None
2675,https://www.martindale.com/attorney/denise-alb...,Denise Alberty,None,None,None,None,Trusts and Estates,None,"Franklin & Marshall College, B.A.; University ...","University of Puerto Rico School of Law, J.D.,...",...,None,None,None,None,None,None,None,None,None,None
2676,https://www.martindale.com/attorney/maria-del-...,María del Pilar Bobonis Zequeira,None,None,None,None,"Litigation, Estates, Commercial",None,"University of Puerto Rico, B.A., 1990","Inter-American University, J.D., 1993",...,None,None,None,None,None,None,None,None,None,None


In [36]:
### Caso 1: Limpieza practice area
practice_cols = [col for col in df.columns if "practice area" in col.lower() and col != "Areas of Practice"]

# 2. Combinar columnas auxiliares en un string limpio
df['combined_practice'] = df[practice_cols].agg(
lambda row: '; '.join(
        str(val).strip()
        for val in row
        if val and str(val).strip().lower() != 'none'),
    axis=1)

# 3. Usar valor combinado solo si no está vacío
df['Areas of Practice'] = df.apply(
    lambda row: row['combined_practice'] if row['combined_practice'] else row['Areas of Practice'],
    axis=1)

# 4. Limpiar y estandarizar el texto final
df['Areas of Practice'] = (
    df['Areas of Practice']
    .fillna('')                                  # manejar posibles NaN
    .str.replace(',', ';')                       # reemplazar , por ;
    .str.replace(r'\s*;\s*', '; ', regex=True)   # limpiar espacios
    .str.strip('; ')                             # quitar ; sobrantes
    .apply(lambda x: '; ' + x if x and not x.startswith(';') else x))

# 5. Eliminar columnas auxiliares
df.drop(columns=practice_cols + ['combined_practice'], inplace=True)
df.head(20)

,url_martindale,name,firm,url_firm_martindale,website_firm,Biography,Areas of Practice,Contact Information,University Attended,Law School Attended,...,Hobbies & interests,Video Call Options,website,Affiliations,Transactions,Mailing Address,Reported Cases,Certifications,Special Agencies,Responsibilities
0,https://www.martindale.com/attorney/ramon-mell...,Ramón Mellado-González,Mellado & Mellado Villarreal,https://www.martindale.com/organization/mellad...,http://www.mellado.com,Ramón has been an attorney and notary in Puert...,; Complex Business Litigation; Banks and Banki...,787-650-6994Phone787-767-2645Faxwww.mellado.co...,"University of Puerto Rico, B.A., Political Sci...","University of Puerto Rico, LL.B., 1958",...,None,None,None,None,None,None,None,None,None,None
1,https://www.martindale.com/attorney/maximilian...,Maximiliano A. Plá-Méndez,"Fernández, Cuyar, Rovira & Plá, LLC",https://www.martindale.com/organization/fernan...,http://fclawpr.com,Maximiliano “Max” A. Plá-Méndez is a member of...,; Contract Law; Notarial Law; Bankruptcy; Coll...,939-296-3363Phonefclawpr.com/our-attorneys/max...,"Universidad del Sagrado Corazón, B.A., 1998; I...","Interamerican University of Puerto Rico, J.D.,...",...,None,None,None,None,None,None,None,None,None,None
2,https://www.martindale.com/attorney/juan-a-cuy...,Juan A. Cuyar-Cobb,"Fernández, Cuyar, Rovira & Plá, LLC",https://www.martindale.com/organization/fernan...,http://fclawpr.com,Juan A. Cuyar-Cobb is a founding member of Fer...,; Contract Law; Notarial Law; Corporate Law; T...,939-296-3363Phonefclawpr.com/our-attorneys/cuyar,"University of Puerto Rico, B.A., magna cum lau...","University of Puerto Rico, J.D., 1995",...,None,None,None,None,None,None,None,None,None,None
3,https://www.martindale.com/attorney/edgardo-j-...,"Edgardo J. Hernández Ohárriz, Esq.","Hernandez - Oharriz & Santiago Law Firm, P.S.C.",https://www.martindale.com/organization/hernan...,http://www.hoslawfirm.com,Partner/co-founder of Hernandez-Oharriz & Sant...,; Personal Injury; Pedestrian Injuries; Plaint...,855-789-8177Phone787-775-1672Faxwww.hoslawfirm...,"University of Puerto Rico, B.B.A., 2007; Unive...","Inter-American University, J.D., 2010",...,None,None,None,None,None,None,None,None,None,None
4,https://www.martindale.com/attorney/roberto-c-...,Roberto C. Rodriguez Poventud,"Bobonis, Bobonis & Rodriguez Poventud",https://www.martindale.com/organization/boboni...,https://bobonislaw.com/,"Author, PRMA's Position Paper with respect to ...",; Tax Law (Federal; State and Municipal Taxes)...,None,"Holy Cross College, B.A. Economics, 1973","Boston University, LL.M., Taxation, 1977; Univ...",...,None,None,None,None,None,None,None,None,None,None
5,https://www.martindale.com/attorney/joseph-d-l...,Joseph D. Lento,Lento Law Group,https://www.martindale.com/organization/lento-...,https://www.lentolawgroup.com,Joseph D. Lento is a dedicated attorney with n...,; Personal Injury; Wrongful Death Accidents; U...,215-791-8762PhoneStudentDisciplineDefense.com,None,Temple University - James E. Beasley School of...,...,None,None,None,None,None,None,None,None,None,None
6,https://www.martindale.com/attorney/jairo-jay-...,"Jairo ""Jay"" Mellado-Villarreal",Mellado & Mellado Villarreal,https://www.martindale.com/organization/mellad...,http://www.mellado.com,Jay Mellado-Villarreal is a litigator with ove...,; Business Law; Bankruptcy; Civil Practice; Co...,787-650-6994Phone787-767-2645Faxwww.mellado.co...,"Boston University, B.A., Political Science, 1989","Interamerican University of Puerto Rico, J.D.,...",...,None,None,None,None,None,None,None,None,None,None
7,https://www.martindale.com/attorney/ms-helaine...,Ms. Helaine A. Gregory,Helaine A. Gregory,https://www.martindale.com/organization/helain...,https://www.linkedin.com/in/helaine-gregory-68...,Helaine Gregory is a diversified professional ...,; Health Care; Government Affairs; Regulatory ...,939-297-2014Phonewww.linkedin.com/in/helaine-g...,"Harvard University, A.B., Sociolo

In [37]:
### Caso 2: Limpieza contact info
list = df["Contact Information"].tolist()
list


['787-650-6994Phone787-767-2645Faxwww.mellado.com/ramon-mellado-gonzalez',
 '939-296-3363Phonefclawpr.com/our-attorneys/maximiliano-pla',
 '939-296-3363Phonefclawpr.com/our-attorneys/cuyar',
 '855-789-8177Phone787-775-1672Faxwww.hoslawfirm.com',
 None,
 '215-791-8762PhoneStudentDisciplineDefense.com',
 '787-650-6994Phone787-767-2645Faxwww.mellado.com/jairo-jay-mellado-villarreal',
 '939-297-2014Phonewww.linkedin.com/in/helaine-gregory-68ba5ab',
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None

In [38]:
def parse_contact_info(text):
    if not isinstance(text, str):
        return pd.Series([[], []], index=["Phones", "Faxes"])

    # Regex para teléfonos: busca 3 dígitos - 3 dígitos - 4 dígitos
    phone_fax_pattern = re.findall(r'(\d{3}-\d{3}-\d{4})(Phone|Fax)?', text)
    
    phones = []
    faxes = []

    # Clasificar como Phone o Fax si viene etiquetado
    i = 0
    while i < len(phone_fax_pattern):
        number, label = phone_fax_pattern[i]
        if label == 'Phone':
            phones.append(number)
        elif label == 'Fax':
            faxes.append(number)
        else:
            # Si no hay etiqueta, asumimos Phone si no está duplicado
            if number not in phones and number not in faxes:
                phones.append(number)
        i += 1

    # Regex para extraer links: comienzan por www. o contienen .com/.org/etc
    ##links = re.findall(r'(?:https?://)?(?:www\.)?[^\s]+(?:\.com|\.org|\.net|\.gov|\.edu|linkedin\.com)[^\s]*', text) Ya se obtuvieron en otra seccion

    return pd.Series([phones, faxes], index=["Phones", "Faxes"])

In [39]:
df[['Phones', 'Faxes']] = df['Contact Information'].apply(parse_contact_info)
df.drop('Contact Information', axis=1, inplace=True)
df.head(4)

,url_martindale,name,firm,url_firm_martindale,website_firm,Biography,Areas of Practice,University Attended,Law School Attended,Year of First Admission,...,website,Affiliations,Transactions,Mailing Address,Reported Cases,Certifications,Special Agencies,Responsibilities,Phones,Faxes
0,https://www.martindale.com/attorney/ramon-mell...,Ramón Mellado-González,Mellado & Mellado Villarreal,https://www.martindale.com/organization/mellad...,http://www.mellado.com,Ramón has been an attorney and notary in Puert...,; Complex Business Litigation; Banks and Banki...,"University of Puerto Rico, B.A., Political Sci...","University of Puerto Rico, LL.B., 1958",1958,...,None,None,None,None,None,None,None,None,[787-650-6994],[787-767-2645]
1,https://www.martindale.com/attorney/maximilian...,Maximiliano A. Plá-Méndez,"Fernández, Cuyar, Rovira & Plá, LLC",https://www.martindale.com/organization/fernan...,http://fclawpr.com,Maximiliano “Max” A. Plá-Méndez is a member of...,; Contract Law; Notarial Law; Bankruptcy; Coll...,"Universidad del Sagrado Corazón, B.A., 1998; I...","Interamerican University of Puerto Rico, J.D.,...",2005,...,None,None,None,None,None,None,None,None,[939-296-3363],[]
2,https://www.martindale.com/attorney/juan-a-cuy...,Juan A. Cuyar-Cobb,"Fernández, Cuyar, Rovira & Plá, LLC",https://www.martindale.com/organization/fernan...,http://fclawpr.com,Juan A. Cuyar-Cobb is a founding member of Fer...,; Contract Law; Notarial Law; Corporate Law; T...,"University of Puerto Rico, B.A., magna cum lau...","University of Puerto Rico, J.D., 1995",1996,...,None,None,None,None,None,None,None,None,[939-296-3363],[]
3,https://www.martindale.com/attorney/edgardo-j-...,"Edgardo J. Hernández Ohárriz, Esq.","Hernandez - Oharriz & Santiago Law Firm, P.S.C.",https://www.martindale.com/organization/hernan...,http://www.hoslawfirm.com,Partner/co-founder of Hernandez-Oharriz & Sant...,; Personal Injury; Pedestrian Injuries; Plaint...,"University of Puerto Rico, B.B.A., 2007; Unive...","Inter-American University, J.D., 2010",2011,...,None,None,None,None,None,None,None,None,[855-789-8177],[787-775-1672]


In [40]:
df.to_excel("Martindale_attorneys_clean.xlsx",index=False)
df.head()

,url_martindale,name,firm,url_firm_martindale,website_firm,Biography,Areas of Practice,University Attended,Law School Attended,Year of First Admission,...,website,Affiliations,Transactions,Mailing Address,Reported Cases,Certifications,Special Agencies,Responsibilities,Phones,Faxes
0,https://www.martindale.com/attorney/ramon-mell...,Ramón Mellado-González,Mellado & Mellado Villarreal,https://www.martindale.com/organization/mellad...,http://www.mellado.com,Ramón has been an attorney and notary in Puert...,; Complex Business Litigation; Banks and Banki...,"University of Puerto Rico, B.A., Political Sci...","University of Puerto Rico, LL.B., 1958",1958,...,None,None,None,None,None,None,None,None,[787-650-6994],[787-767-2645]
1,https://www.martindale.com/attorney/maximilian...,Maximiliano A. Plá-Méndez,"Fernández, Cuyar, Rovira & Plá, LLC",https://www.martindale.com/organization/fernan...,http://fclawpr.com,Maximiliano “Max” A. Plá-Méndez is a member of...,; Contract Law; Notarial Law; Bankruptcy; Coll...,"Universidad del Sagrado Corazón, B.A., 1998; I...","Interamerican University of Puerto Rico, J.D.,...",2005,...,None,None,None,None,None,None,None,None,[939-296-3363],[]
2,https://www.martindale.com/attorney/juan-a-cuy...,Juan A. Cuyar-Cobb,"Fernández, Cuyar, Rovira & Plá, LLC",https://www.martindale.com/organization/fernan...,http://fclawpr.com,Juan A. Cuyar-Cobb is a founding member of Fer...,; Contract Law; Notarial Law; Corporate Law; T...,"University of Puerto Rico, B.A., magna cum lau...","University of Puerto Rico, J.D., 1995",1996,...,None,None,None,None,None,None,None,None,[939-296-3363],[]
3,https://www.martindale.com/attorney/edgardo-j-...,"Edgardo J. Hernández Ohárriz, Esq.","Hernandez - Oharriz & Santiago Law Firm, P.S.C.",https://www.martindale.com/organization/hernan...,http://www.hoslawfirm.com,Partner/co-founder of Hernandez-Oharriz & Sant...,; Personal Injury; Pedestrian Injuries; Plaint...,"University of Puerto Rico, B.B.A., 2007; Unive...","Inter-American University, J.D., 2010",2011,...,None,None,None,None,None,None,None,None,[855-789-8177],[787-775-1672]
4,https://www.martindale.com/attorney/roberto-c-...,Roberto C. Rodriguez Poventud,"Bobonis, Bobonis & Rodriguez Poventud",https://www.martindale.com/organization/boboni...,https://bobonislaw.com/,"Author, PRMA's Position Paper with respect to ...",; Tax Law (Federal; State and Municipal Taxes)...,"Holy Cross College, B.A. Economics, 1973","Boston University, LL.M., Taxation, 1977; Univ...",1976,...,None,None,None,None,None,None,None,None,[],[]
